In [5]:
from scipy.optimize import least_squares
import numpy as np

In [1]:
# Define a function for matrix multiplication
def matrix_multiply(A, B):
    return np.dot(A, B)

# mpcgain function
def mpcgain(Ad, Bd, Cd, Nc, Np):
    
    # Augmented Discrete-time model of Ad, Bd, Cd matrix
    N_outputs, N_states = Cd.shape
    N_states, N_inputs = Bd.shape
    
    # A 만들기
    #          Bd행(=Ad행)  Cd행
    Ae = np.eye(N_states + N_outputs)
    Ae[:N_states, :N_states] = Ad
    Ae[N_states:, :N_states] = np.dot(Cd, Ad)
    
    # B 만들기
    #              Bd행       Cd행       Bd열
    Be = np.zeros((N_states + N_outputs, N_inputs))
    Be[:N_states, :] = Bd
    Be[N_states:, :] = np.dot(Cd, Bd)
    
    # C 만들기
    #              Cd행       Cd열       Cd행
    Ce = np.zeros((N_outputs, N_states + N_outputs))
    #Ce[:, :N_states] = Cd
    Ce[:, N_states:] = np.eye(N_outputs)
    
    # h는 Phi만들때 사용한다.
    h = Ce
    # 이놈은 그냥 F 다.
    F = Ce @ Ae

    for k in range(2, Np + 1):
        h = np.concatenate([h, h[-Ce.shape[0]:, :]@Ae], axis=0)
        F = np.concatenate([F, (F[-Ce.shape[0]:]@ Ae)], axis=0) 

    v = Ce@Be
    for i in range(1, Np):
        v = np.concatenate([v, h[Cd.shape[0]*i:Cd.shape[0]*(i+1)]@Be], axis=0)


    Phi = v
    for i in range(1, Nc):
        Phi = np.concatenate([Phi, np.concatenate([np.zeros((Ce.shape[0]*i, Be.shape[1])), v[:Ce.shape[0]*(Np-i)]], axis=0)], axis=1)
    
    
    #Phi = np.zeros((Np, Nc))
    #Phi[:, 0] = v[:, 0].reshape(Phi[:, 0].shape) # first column of Phi

    #for i in range(1, Nc):
    #    Phi[:, i] = np.concatenate([np.zeros(i), v[:Np - i, 0]], axis=0).reshape(Phi[:, i].shape)

    #Rs = r[:Np]   
    #Phi_T_F = matrix_multiply(Phi.T, F)
    #Phi_T_R = matrix_multiply(Phi.T, Rs)
    
    return Phi, F, Ae, Be, Ce

In [2]:
def fun(DeltaU, r, F, xf, Phi, R_bar):
    DeltaU = DeltaU.reshape(-1, 1)
    Y = F@xf + Phi @ DeltaU
    return ((r-Y).T @ (r-Y) + DeltaU.T @ R_bar @ DeltaU)[0]

In [3]:
def jac(DeltaU, r, F, xf, Phi, R_bar):
    DeltaU = DeltaU.reshape(-1, 1)
    a = -2 * Phi.T @ (r - F@xf) + 2*(Phi.T@Phi + R_bar)@DeltaU
    return a.T